In [1]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm

from geopy.distance import geodesic

%matplotlib inline
import mplleaflet
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.pylab as pylab
import seaborn as sns

import os
import warnings

from k_medoids import KMedoidsWeighted
import helper as hp

warnings.simplefilter("ignore")
tqdm.pandas()

np.random.seed(3)

In [2]:
data_dir = '../data/' # Diretório dos arquivos
quimio_name = 'Pulmão Quimioterapia SIA-SUS.csv' # Tipo de câncer a ser analisado (quimioterapia)
radio_name = 'Pulmão Radioterapia SIA-SUS.csv' # Tipo de câncer a ser analisado (radioterapia)
municipios_name = 'dados_municipios.csv'  #Dados dos municípios
area_name = 'dados_area.csv' #Área dos estados

dtype_cols = {'AP_MUNPCN': str, 'AP_UFMUN': str}
usecols = ['AP_MUNPCN', 'AP_UFMUN']

In [3]:
# Lendo os dataframes:

quimio = pd.read_csv(os.path.join(data_dir, quimio_name), encoding='latin', 
                     error_bad_lines=False, low_memory=False, dtype=dtype_cols, usecols=usecols)

radio = pd.read_csv(os.path.join(data_dir, radio_name), encoding='latin',
                    error_bad_lines=False, low_memory=False, dtype=dtype_cols, usecols=usecols)

municipios = pd.read_csv(os.path.join(data_dir, municipios_name), index_col=0, dtype={'COD_MUN': str})

area =  pd.read_csv(os.path.join(data_dir, area_name), encoding='latin', sep = ';', 
                    index_col=0, dtype={'COD': str})

In [4]:
# Removendo o último digito para se adequar ao dataframe das APACs:

municipios.index = municipios['COD_MUN'].apply(lambda x: x[:-1])

In [5]:
cancer = pd.concat([quimio, radio])

In [6]:
cancer['DIST'] = cancer.progress_apply(lambda row: hp.calculte_distance(row, municipios), axis=1)

100%|██████████| 330771/330771 [01:23<00:00, 3966.92it/s]


In [8]:
#Criando um dicionario entre os estados e os códigos:

lista_codigo_uf = municipios['CODIGO_UF'].unique().astype(str) 
lista_nome_uf = municipios['UF'].unique().astype(str)

#Remover 'DF'
lista_nome_uf = np.delete(lista_nome_uf, np.where(lista_nome_uf == 'DF'))
nome_to_codigo_uf = dict(zip(lista_nome_uf, lista_codigo_uf))
codigo_to_nome_uf = dict(zip(lista_codigo_uf, lista_nome_uf))

In [12]:
locais_de_tratamento = {}
distancias = {}
indicadores = {}
for uf in lista_nome_uf:
    codigo_uf = str(nome_to_codigo_uf[uf])
    cancer_estado = hp.generate_df_estado(cancer, municipios, codigo_uf)
    locais_de_tratamento[codigo_uf] = len(cancer_estado['AP_UFMUN'].unique())
    distancias[codigo_uf] = cancer_estado[cancer_estado['DIST']>0]['DIST'].mean()
    sqrtA = np.sqrt(area.loc[int(codigo_uf)]['ÁREA'])
    indicadores[codigo_uf] = distancias[codigo_uf]/sqrtA

In [13]:
df = pd.DataFrame({'Número de centros de tratamento': locais_de_tratamento, 
     'Distância média': distancias, 
     'Índice': indicadores, 'Nome': codigo_to_nome_uf})
df.sort_values(by=['Índice'], inplace = True)

In [14]:
df

,Número de centros de tratamento,Distância média,Índice,Nome
16,1,35.760356,0.094623,AP
31,24,79.073251,0.103249,MG
15,2,126.236474,0.113002,PA
43,21,62.078862,0.116957,RS
42,13,39.484984,0.127613,SC
35,40,71.958369,0.144431,SP
41,15,73.923461,0.165585,PR
33,12,42.126006,0.201332,RJ
51,3,191.885101,0.201887,MT
29,7,158.794796,0.211307,BA
